In [1]:
using Knet,Compat,Interact,Plots;
include("../julia/test_train_split.jl");

In [2]:
# file, training_proportion, num_input_stops, stops_ahead_to_predict, stops_to_predict
#routes = get_routes("/Users/Cooper/Desktop/mbta_trajectories_2014_13.json",0.6,9,0,1,[],false);
routes = get_routes("/Users/Cooper/Desktop/mbta_trajectories_2014_13.json",0.8,5,0,1,["day_of_week","bus_id","hour"],true);
# routes = get_routes("../python/full_routes.mat",0.8,10,1,2);

x_train  = routes["train_input_data"];
y_train = routes["train_output_data"];
x_test = routes["test_input_data"];
y_test = routes["test_output_data"];


5
1
(129, 323)
(1, 323)


In [3]:
function predict(w,x)
    for i=1:2:length(w)
        # w is a vector of weight matrices
        x = w[i]*x .+ w[i+1]
        if i<length(w)-1
            # Apply ReLU nonlinearity
            x = max.(0,x)
        end
    end
    return x
end

loss(w,x,y)=0.5*(sum((y-predict(w,x)).^2) / size(x,2))

lossgradient = grad(loss);

function train(w, dtrn; lr=.5, epochs=10)
    for epoch=1:epochs
        for (x,y) in dtrn
            g = lossgradient(w, x, y)
            for i in 1:length(w)
                axpy!(-lr, g[i], w[i])
            end
        end
    end
    return w
end

function weights(h)
    w = Any[]
    x = size(x_train)[1]
    for y in vcat(h,size(y_train)[1])
        push!(w,0.1*randn(y,x))
        push!(w, zeros(y, 1))
        x = y
    end
    return w
end


function accuracy(w, dtst, pred=predict)
    s = 0
    n = 0
    s_resid = 0
    yvar = var(y_test)
    for (x, ygold) in dtst
        ypred = pred(w, x)
        s+= (ypred - ygold).^2
        n+=1
    end
    s_tot =yvar*n
    return (sqrt.(s/n),1-(s/s_tot))
end

accuracy (generic function with 2 methods)

In [4]:
function main()
    hidden_layer_sizes = [100,100]
    w = weights(hidden_layer_sizes)
    dtrn = zip([x_train[:,i] for i in 1:size(x_train,2)],y_train)
    dtst = zip([x_test[:,i] for i in 1:size(x_test,2)],y_test)
    report(epoch)=println((epoch,"Train MAE: ",accuracy(w,dtrn),"Test MAE: ",accuracy(w,dtst)))
    report(0)
    epochs = 100
    learning_rate = 0.00001
    @time for epoch=1:epochs
        train(w, dtrn; lr=learning_rate, epochs=1)
        report(epoch)
#       gradcheck(loss, w, first(dtrn)...; gcheck=o[:gcheck], verbose=true)
    end
    return w
end

main();

(0, "Train MAE: ", ([75.7913], [-1.20398]), "Test MAE: ", ([83.426], [-1.67037]))
(1, "Train MAE: ", ([75.3822], [-1.18025]), "Test MAE: ", ([83.0021], [-1.6433]))
(2, "Train MAE: ", ([74.9725], [-1.15662]), "Test MAE: ", ([82.5775], [-1.61633]))
(3, "Train MAE: ", ([74.4731], [-1.12798]), "Test MAE: ", ([82.0604], [-1.58367]))
(4, "Train MAE: ", ([73.7484], [-1.08677]), "Test MAE: ", ([81.3176], [-1.5371]))
(5, "Train MAE: ", ([72.5413], [-1.01902]), "Test MAE: ", ([80.0827], [-1.46063]))
(6, "Train MAE: ", ([70.262], [-0.894131]), "Test MAE: ", ([77.7707], [-1.3206]))
(7, "Train MAE: ", ([65.5489], [-0.64854]), "Test MAE: ", ([73.0324], [-1.04645]))
(8, "Train MAE: ", ([56.4735], [-0.223654]), "Test MAE: ", ([63.9442], [-0.568816]))
(9, "Train MAE: ", ([47.287], [0.142069]), "Test MAE: ", ([54.416], [-0.136116]))
(10, "Train MAE: ", ([44.6255], [0.235927]), "Test MAE: ", ([51.4295], [-0.0148308]))
(11, "Train MAE: ", ([44.0614], [0.255123]), "Test MAE: ", ([51.1002], [-0.0018779]))
(

(96, "Train MAE: ", ([22.0717], [0.813087]), "Test MAE: ", ([52.7235], [-0.0665405]))
(97, "Train MAE: ", ([21.7151], [0.819078]), "Test MAE: ", ([52.7519], [-0.0676887]))
(98, "Train MAE: ", ([21.3645], [0.824872]), "Test MAE: ", ([52.7396], [-0.0671901]))
(99, "Train MAE: ", ([20.9857], [0.831027]), "Test MAE: ", ([52.7919], [-0.0693082]))
(100, "Train MAE: ", ([20.6248], [0.836789]), "Test MAE: ", ([52.6945], [-0.0653692]))
 35.717099 seconds (32.17 M allocations: 6.412 GiB, 3.28% gc time)


In [ ]:
x_train[:,2]

In [ ]:
# Plot the trajectory
@manipulate for i = 1:10
plot(cumsum(x_train[:,i]))
end


In [ ]:
hidden_layer_sizes = [20,10,1]
w1 = weights(hidden_layer_sizes)
x = [1 2 3 4 5 6 7 8 9 10]
predict(w1,x')

In [ ]:
histogram(x_train[15,:])

In [ ]:
histogram(y_train')
# mean(y_train)
# sqrt(var(y_train))

In [ ]:
# https://arxiv.org/pdf/1407.5949.pdf
function lstm(weight,bias,hidden,cell,input)
    gates   = hcat(input,hidden) * weight .+ bias
    hsize   = size(hidden,2)
    forget  = sigm(gates[:,1:hsize])
    ingate  = sigm(gates[:,1+hsize:2hsize])
    outgate = sigm(gates[:,1+2hsize:3hsize])
    change  = tanh(gates[:,1+3hsize:end])
    cell    = cell .* forget + ingate .* change
    hidden  = outgate .* tanh(cell)
    return (hidden,cell)
end

function initmodel(atype, hidden, vocab, embed)
    init(d...)=atype(xavier(d...))
    
    bias(d...)=atype(zeros(d...))
    model = Array(Any, 2*length(hidden)+3)
    X = embed
    for k = 1:length(hidden)
        H = hidden[k]
        model[2k-1] = init(X+H, 4H)
        model[2k] = bias(1, 4H)
        model[2k][1:H] = 1 # forget gate bias = 1
        X = H
    end
    model[end-2] = init(vocab,embed)
    model[end-1] = init(hidden[end],vocab)
    model[end] = bias(1,vocab)
    return model
end

let blank = nothing; global initstate
function initstate(model, batch)
    nlayers = div(length(model)-3,2)
    state = Array(Any, 2*nlayers)
    for k = 1:nlayers
        bias = model[2k]
        hidden = div(length(bias),4)
        if typeof(blank)!=typeof(bias) || size(blank)!=(batch,hidden)
            blank = fill!(similar(bias, batch, hidden),0)
        end
        state[2k-1] = state[2k] = blank
    end
    return state
end
end

function predict(model, state, input; pdrop=0)
    nlayers = div(length(model)-3,2)
    newstate = similar(state)
    for k = 1:nlayers
        input = dropout(input, pdrop)
        
        (newstate[2k-1],newstate[2k]) = lstm(model[2k-1],model[2k],state[2k-1],state[2k],input)
        input = newstate[2k-1]
    end
    return input,newstate
end